In [ ]:
import geopandas as gpd
import folium
import panel as pn
import pandas as pd
import numpy as np
from pygris import tracts


pn.extension()

# Load all tracts in the US
all_tracts = tracts(cb=True, year=2020, cache=True)

#Load EJScreen Data
life_exp_tracts = pd.read_csv("EJScreenNLifeExp_Col_Names.csv",
                          dtype = {"tract_fips": "object"})


Retrieving Census tracts for the entire United States


In [ ]:
# Merge the EJScreen data with the geospatial data for the tracts

all_tracts_merged = all_tracts.merge(life_exp_tracts, how='left', left_on="GEOID", right_on="tract_fips")

print("Status: Merge Completed")



Status: Merge Completed


In [ ]:

#Import Random Forest Model to be used during the interactive portion of the Visualization
import pickle
filename = 'rf_model.pkl'
with open(filename, 'rb') as file:
    rf, top_features = pickle.load(file)

#Top 10 features from Random Forest
#VULEOPCT
#LINGISOPCT
#LOWINCPCT
#PRE1960PCT
#LESSHSPCT
#DSLPM
#UST
#UNEMPPCT
#OVER64PCT
#PM25

# Set Map Parameters
explore_params = dict(
    column = "AdjLifeExpec",
    cmap='RdYlGn',
    vmin=65,
    vmax=90,
    tooltip=False,
    popup=True,
    tiles="CartoDB positron",
    style_kwds=dict(weight=0.7, nan_fill_color="lightgrey"),
    #popup_kwds=dict(aliases=["Census tract", "Life Expectancy", "Adjusted Life Expectancy"]),
    legend_kwds=dict(caption="Life Expectancy"),
)
print(top_features)
columns = top_features.tolist() + ['GEOID', 'CalcLifeExpec', 'geometry']
print(columns)
# Define function to create a map for a given state
def create_map(state_abbv):
    # Filter the merged data to only include the columns we need
    state_tracts = all_tracts_merged.loc[all_tracts_merged['STUSPS'] == state_abbv]
    state_tracts = state_tracts.filter(columns)
    state_tracts = state_tracts.assign(AdjLifeExpec=state_tracts['CalcLifeExpec'].copy())

    # Create the map using pygris (folium)
    folium_map = state_tracts.explore(**explore_params)


    # Convert the map to HTML to display in Panel
    html_pane = pn.pane.HTML(height=720, width=1080)
    html_pane.object = folium_map._repr_html_()
    
    return html_pane, state_tracts


Index(['VULEOPCT', 'LINGISOPCT', 'LOWINCPCT', 'PRE1960PCT', 'LESSHSPCT',
       'DSLPM', 'UST', 'UNEMPPCT', 'OVER64PCT', 'PM25'],
      dtype='object')
['VULEOPCT', 'LINGISOPCT', 'LOWINCPCT', 'PRE1960PCT', 'LESSHSPCT', 'DSLPM', 'UST', 'UNEMPPCT', 'OVER64PCT', 'PM25', 'GEOID', 'CalcLifeExpec', 'geometry']


In [ ]:
# Define list of States
state_abbv = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}


state_default = 'DE'  # Default to Delaware (smallest state to run when testing)

#Create the state dropdown
state_maps = {}
state_dropdown = pn.widgets.Select(name="State", options=state_abbv, value=state_default)

# Create the initial map
state_maps[state_default] = create_map(state_default)
map_html = state_maps[state_default][0]



#Initialize all the maps to begin
for state_abbv in state_abbv.values():
    state_maps[state_abbv] = create_map(state_abbv)
    print(state_abbv + " completed")



AL completed
AK completed
AZ completed
AR completed
CA completed
CO completed
CT completed
DE completed
DC completed
FL completed
GA completed
HI completed
ID completed
IL completed
IN completed
IA completed
KS completed
KY completed
LA completed
ME completed
MD completed
MA completed
MI completed
MN completed
MS completed
MO completed
MT completed
NE completed
NV completed
NH completed
NJ completed
NM completed
NY completed
NC completed
ND completed
OH completed
OK completed
OR completed
PA completed
RI completed
SC completed
SD completed
TN completed
TX completed
UT completed
VT completed
VA completed
WA completed
WV completed
WI completed
WY completed


In [ ]:
#Create sliders & checkboes to update life expectancies

text = pn.pane.Markdown("Adjust the following 10 parameters as a % of their \n original values and see it's effect \n on Life Expectancy")

widget1 = pn.widgets.FloatSlider(name='Demographic Index', start=75, end=125, step=2.5, value=100)
widget2 = pn.widgets.FloatSlider(name='Linguistically Isolated', start=75, end=125, step=2.5, value=100)
widget3 = pn.widgets.FloatSlider(name='Low Income', start=75, end=125, step=2.5, value=100)
widget4 = pn.widgets.FloatSlider(name='Lead Paint Indicator (Housing built <1960)', start=75, end=125, step=2.5, value=100)
widget5 = pn.widgets.FloatSlider(name='Less High School', start=75, end=125, step=2.5, value=100)
widget6 = pn.widgets.FloatSlider(name='Diesel Particulate Matter Level', start=75, end=125, step=2.5, value=100)
widget7 = pn.widgets.FloatSlider(name='Underground Storage Tank Proximity', start=75, end=125, step=2.5, value=100)
widget8 = pn.widgets.FloatSlider(name='Unemployment', start=75, end=125, step=2.5, value=100)
widget9 = pn.widgets.FloatSlider(name='Persons over the age of 65', start=75, end=125, step=2.5, value=100)
widget10 = pn.widgets.FloatSlider(name='PM 2.5 Levels in the Air', start=75, end=125, step=2.5, value=100)

recalculate_widget = pn.widgets.Button(
    name='Rerun Predictions', 
    button_type='primary'
)

return_widget = pn.widgets.Button(
    name = 'Reset to Original Life Expectancies',
    button_type='primary'
)

info_widget = pn.widgets.RadioButtonGroup(
    name='More Information about parameters'
)
# Define the function to update the map when the state is changed
def change_state(event):
    state_abbv = state_dropdown.value
    map_html.object = state_maps[state_abbv][0].object
    widget1.value = 100
    widget2.value = 100
    widget3.value = 100
    widget4.value = 100
    widget5.value = 100
    widget6.value = 100
    widget7.value = 100
    widget8.value = 100
    widget9.value = 100
    widget10.value = 100

    print("Changed state to " + state_abbv)



# Define the callback
def recalculate(event):

    state_abbv = state_dropdown.value
    state_tracts = state_maps[state_abbv][1]
    state_tracts_noGeo = state_tracts.dropna()
    # create a boolean mask for non-null values
    not_null_mask = state_tracts['AdjLifeExpec'].notnull()

    # calculate new AdjLifeExpec values only for non-null values
    new_inputs = [state_tracts_noGeo[columns[0]] * widget1.value/100,
                    state_tracts_noGeo[columns[1]] * widget2.value/100,
                    state_tracts_noGeo[columns[2]] * widget3.value/100,
                    state_tracts_noGeo[columns[3]] * widget4.value/100,
                    state_tracts_noGeo[columns[4]] * widget5.value/100,
                    state_tracts_noGeo[columns[5]] * widget6.value/100,
                    state_tracts_noGeo[columns[6]] * widget7.value/100,
                    state_tracts_noGeo[columns[7]] * widget8.value/100,
                    state_tracts_noGeo[columns[8]] * widget9.value/100,
                    state_tracts_noGeo[columns[9]] * widget10.value/100]
    new_inputs = [list(x) for x in zip(*new_inputs)]

    print(new_inputs)
    state_tracts_noGeo['AdjLifeExpec'] = rf.predict(new_inputs)
    state_tracts = state_tracts_noGeo
    folium_map = state_tracts.explore(**explore_params)
    map_html.object = folium_map._repr_html_()
    print("Updated Counts for " + state_abbv)


#Return to original EJScreen Life Expetancies
def reset(event):
    state_abbv = state_dropdown.value
    state_tracts = state_maps[state_abbv][1]
    state_tracts['AdjLifeExpec'] = state_tracts['CalcLifeExpec']
    folium_map = state_tracts.explore(**explore_params)
    map_html.object = folium_map._repr_html_()
    widget1.value = 100
    widget2.value = 100
    widget3.value = 100
    widget4.value = 100
    widget5.value = 100
    widget6.value = 100
    widget7.value = 100
    widget8.value = 100
    widget9.value = 100
    widget10.value = 100   



# Add the callback to the widgets
state_dropdown.param.watch(change_state, 'value')
recalculate_widget.on_click(recalculate)
return_widget.on_click(reset)


# Create the Panel app
template = pn.template.FastListTemplate(
    title="Life Expectancy & Its Contributors by US Census Tract",
    sidebar=[state_dropdown, text, widget1, widget2, widget3, widget4, widget5, widget6, widget7, widget8, widget9, widget10, recalculate_widget, return_widget],
    main=[map_html],
    accent_base_color="#0276ab",
    header_background="#0276ab",
)
template.show()
#template.servable()


Launching server at http://localhost:60681
